### FAZER:
- Puxar o cash amount e spread do btg salvos na base da risktech (api all prices)
- Puxar cod_sys de todos os produtos que temos em posição
- Puxar o cash amount e spread da bbg (no caso de CDS) para todos ativos em posição
- Dar fillna(0) e calcular a dif entre bbg e btg
- Mandar mensagem no chat do gerencial, indicando se está tudo batido ou se existe dif

### Pegar preços BTG

In [1]:
import pandas as pd
import database.queries.data_source as data_source
from pyrisk import days_ago

In [2]:
date = days_ago(1)
date

datetime.date(2023, 1, 4)

In [3]:
endpoint = data_source.Source().risktech_all_prices.format(asset_class='cds', date=date)
endpoint

'http://10.21.2.20/risk-api/all_prices/?asset_class=cds&date=2023-01-04'

In [4]:
risktech_all_prices = pd.read_json(data_source.Source().asset_info.format(product_class='cds'))
risktech_all_prices.head()

,id,ticker_bbg,asset_name,cod_sys,product_class,country,country_code,country_of_risk,currency,exchange_code,asset_class,sec_type,isin,multiplier,figi,cusip,security_id,sedol
0,1,ABEV US EQUITY,AMBEV SA-ADR,ABEV,ADR,BRAZIL,BR,BR,USD,,Equity,ADR,US02319V1035,1.0,BBG000BN5VZ4,02319V103,EQ0038421400009000,BG804F6
1,2,ABEV3 BZ EQUITY,AMBEV SA,ABEV3,Equity,BRAZIL,BR,BR,BRL,,Equity,Common Stock,BRABEVACNOR1,1.0,None,,EQ0038421400002000,BG7ZWY7
2,3,ALPA4 BZ EQUITY,ALPARGATAS SA - PREF,ALPA4,Equity,BRAZIL,BR,BR,BRL,,Equity,Preference,BRALPAACNPR7,1.0,None,,EQ0014820400002000,B07C796
3,4,ALSO3 BZ EQUITY,ALIANSCE SONAE SHOPPING CENT,ALSO3,Equity,BRAZIL,BR,BR,BRL,,Equity,Common Stock,BRALSOACNOR5,1.0,None,,EQ0000000002923563,BJVHGR1
4,5,ALUP11 BZ EQUITY,ALUPAR INVESTIMENTO SA-UNIT,ALUP11,Equity,BRAZIL,BR,BR,BRL,,Equity,Unit,BRALUPCDAM15,1.0,None,,EQ0000000009211806,B8DWT90


In [5]:
risktech_all_prices = pd.read_json(endpoint)
risktech_all_prices.head()

,cod_sys,cash_settled_amnt_admin,px_close_admin,id,ticker_bbg,product_class,cash_settled_amnt,px_close,spread,amount
0,CDS BRAZIL 12/27 ICE,0.065874,258.425000,63483,CBRZ1U5 CORP,CDS,NaN,258.656,100,207500000
1,CDS BRITEL 12/27 ICE,0.018532,143.010000,62079,SPPO0APM CORP,CDS,NaN,4.580,100,25000000
2,CDS CHILE 12/27 ICE,0.004105,110.370000,62206,CCHIL1U5 CORP,CDS,NaN,109.678,100,75000000
3,CDS FCO 12/27 ICE,-0.059980,353.900000,61463,CFM1U5 CORP,CDS,NaN,360.733,500,60000000
4,CDS GM 06/27 ICE,-0.115692,203.596164,53784,CY075977 CORP,CDS,NaN,217.517,500,45000000


In [6]:
if risktech_all_prices[pd.isnull(risktech_all_prices['px_close_admin'])].empty:
    print("Não existem CDS sem preço do BTG")
else:
    print(risktech_all_prices[pd.isnull(risktech_all_prices['px_close_admin'])])

Não existem CDS sem preço do BTG


In [7]:
if risktech_all_prices[pd.isnull(risktech_all_prices['ticker_bbg'])].empty:
    print("Não existem CDS sem ticker")
else:
    print(risktech_all_prices[pd.isnull(risktech_all_prices['ticker_bbg'])])

                cod_sys  cash_settled_amnt_admin  px_close_admin  id  \
7   CDS TESCO 12/27 ICE                 0.017665          140.56 NaN   
8  CDS VODFON 12/27 ICE                -0.001194           97.90 NaN   

  ticker_bbg product_class  cash_settled_amnt  px_close  spread    amount  
7       None          None                NaN       NaN     NaN  50000000  
8       None          None                NaN       NaN     NaN  25000000  


In [8]:
btg_prices = risktech_all_prices[['cod_sys', 'ticker_bbg', 'cash_settled_amnt_admin', 'px_close_admin']]
btg_prices = btg_prices.rename(columns={'ticker_bbg': 'ticker'})

In [9]:
btg_prices.head()

,cod_sys,ticker,cash_settled_amnt_admin,px_close_admin
0,CDS BRAZIL 12/27 ICE,CBRZ1U5 CORP,0.065287,256.005000
1,CDS BRITEL 12/27 ICE,SPPO0APM CORP,0.024657,156.755000
2,CDS CHILE 12/27 ICE,CCHIL1U5 CORP,0.003358,108.195000
3,CDS FCO 12/27 ICE,CFM1U5 CORP,-0.058281,355.995000
4,CDS GM 06/27 ICE,CY075977 CORP,-0.115541,202.554685


In [10]:
btg_prices[pd.isnull(btg_prices['ticker'])]

,cod_sys,ticker,cash_settled_amnt_admin,px_close_admin
7,CDS TESCO 12/27 ICE,None,0.017665,140.56
8,CDS VODFON 12/27 ICE,None,-0.001194,97.90


In [11]:
cds_list = btg_prices['ticker'].tolist()

In [12]:
import database.get_data as gd

In [13]:
bbg_price = gd.get_cds_list_price(cds_list, date)

In [ ]:
bbg_price = bbg_price[bbg_price['field'] != 'CDS_FLAT_SPREAD']

In [ ]:
bbg_price.head()

,ticker,field,value
0,CXPEM538 CORP,CDS_CASH_SETTLED_AMOUNT,582306.0
2,CXPEM538 CORP,CDS_REPL,242.198631
3,CXPHY539 CORP,CDS_CASH_SETTLED_AMOUNT,-165782.0
5,CXPHY539 CORP,CDS_REPL,488.848324
6,ITXEB538 CORP,CDS_CASH_SETTLED_AMOUNT,-37422.0


In [ ]:
result_df = pd.merge(bbg_price, btg_prices, how='inner', on='ticker') 

In [ ]:
result_df

,ticker,field,value,cod_sys,cash_settled_amnt_admin,px_close_admin
0,CXPEM538 CORP,CDS_CASH_SETTLED_AMOUNT,582306.0,CDX.EM.38 12/27 ICE,0.054245,232.281852
1,CXPEM538 CORP,CDS_REPL,242.198631,CDX.EM.38 12/27 ICE,0.054245,232.281852
2,CXPHY539 CORP,CDS_CASH_SETTLED_AMOUNT,-165782.0,CDX.NA.HY.39 12/27 ICE,-0.022052,474.615174
3,CXPHY539 CORP,CDS_REPL,488.848324,CDX.NA.HY.39 12/27 ICE,-0.022052,474.615174
4,ITXEB538 CORP,CDS_CASH_SETTLED_AMOUNT,-37422.0,ITRX EUR S38 12/27 ICE,-0.007059,89.906400
5,ITXEB538 CORP,CDS_REPL,97.1536,ITRX EUR S38 12/27 ICE,-0.007059,89.906400
6,ITXES538 CORP,CDS_CASH_SETTLED_AMOUNT,6152.0,ITRX EUR S38 SNR FIN 12/27 ICE,-0.002558,99.690700
7,ITXES538 CORP,CDS_REPL,106.7368,ITRX EUR S38 SNR FIN 12/27 ICE,-0.002558,99.690700
8,CBRZ1U5 CORP,CDS_CASH_SETTLED_AMOUNT,638272.0,CDS BRAZIL 12/27 ICE,0.062295,252.030000
9,CBRZ1U5 CORP,CDS_REPL,256.0,CDS BRAZIL 12/27 ICE,0.062295,252.030000


In [ ]:
result_df.loc[result_df['field']=='CDS_REPL', ['dif']] = result_df['value'] - result_df['px_close_admin']
result_df.loc[result_df['field']=='CDS_REPL', ['dif_bps']] = result_df['dif'] * 100000 / (result_df['value'])

result_df.loc[result_df['field']=='CDS_CASH_SETTLED_AMOUNT', ['dif']] = (result_df['value']/10000000) - result_df['cash_settled_amnt_admin']
result_df.loc[result_df['field']=='CDS_CASH_SETTLED_AMOUNT', ['dif_bps']] = result_df['dif'] * 100000 / (result_df['value'])

dif = bbg - btg (correto - btg)

In [ ]:
error_df = result_df[result_df['dif_bps'].abs() > 1]

In [ ]:
result_df

,ticker,field,value,cod_sys,cash_settled_amnt_admin,px_close_admin,dif,dif_bps
0,CXPEM538 CORP,CDS_CASH_SETTLED_AMOUNT,582306.0,CDX.EM.38 12/27 ICE,0.054245,232.281852,0.003985,0.000684
1,CXPEM538 CORP,CDS_REPL,242.198631,CDX.EM.38 12/27 ICE,0.054245,232.281852,9.916779,4094.481781
2,CXPHY539 CORP,CDS_CASH_SETTLED_AMOUNT,-165782.0,CDX.NA.HY.39 12/27 ICE,-0.022052,474.615174,0.005474,-0.003302
3,CXPHY539 CORP,CDS_REPL,488.848324,CDX.NA.HY.39 12/27 ICE,-0.022052,474.615174,14.233150,2911.567654
4,ITXEB538 CORP,CDS_CASH_SETTLED_AMOUNT,-37422.0,ITRX EUR S38 12/27 ICE,-0.007059,89.906400,0.003317,-0.008864
5,ITXEB538 CORP,CDS_REPL,97.1536,ITRX EUR S38 12/27 ICE,-0.007059,89.906400,7.247200,7459.528005
6,ITXES538 CORP,CDS_CASH_SETTLED_AMOUNT,6152.0,ITRX EUR S38 SNR FIN 12/27 ICE,-0.002558,99.690700,0.003174,0.051586
7,ITXES538 CORP,CDS_REPL,106.7368,ITRX EUR S38 SNR FIN 12/27 ICE,-0.002558,99.690700,7.046100,6601.378344
8,CBRZ1U5 CORP,CDS_CASH_SETTLED_AMOUNT,638272.0,CDS BRAZIL 12/27 ICE,0.062295,252.030000,0.001532,0.000240
9,CBRZ1U5 CORP,CDS_REPL,256.0,CDS BRAZIL 12/27 ICE,0.062295,252.030000,3.970000,1550.781250


In [ ]:
error_df 

,ticker,field,value,cod_sys,cash_settled_amnt_admin,px_close_admin,dif,dif_bps
1,CXPEM538 CORP,CDS_REPL,242.198631,CDX.EM.38 12/27 ICE,0.054245,232.281852,9.916779,4094.481781
3,CXPHY539 CORP,CDS_REPL,488.848324,CDX.NA.HY.39 12/27 ICE,-0.022052,474.615174,14.233150,2911.567654
5,ITXEB538 CORP,CDS_REPL,97.1536,ITRX EUR S38 12/27 ICE,-0.007059,89.906400,7.247200,7459.528005
7,ITXES538 CORP,CDS_REPL,106.7368,ITRX EUR S38 SNR FIN 12/27 ICE,-0.002558,99.690700,7.046100,6601.378344
9,CBRZ1U5 CORP,CDS_REPL,256.0,CDS BRAZIL 12/27 ICE,0.062295,252.030000,3.970000,1550.781250
11,SPPO0APM CORP,CDS_REPL,151.74,CDS BRITEL 12/27 ICE,0.018063,145.545000,6.195000,4082.641360
13,CCHIL1U5 CORP,CDS_REPL,107.0,CDS CHILE 12/27 ICE,0.000305,106.100000,0.900000,841.121495
15,CFM1U5 CORP,CDS_REPL,336.865,CDS FCO 12/27 ICE,-0.081075,329.520000,7.345000,2180.398676
17,CY075977 CORP,CDS_REPL,210.465,CDS GM 06/27 ICE,-0.133872,188.382068,22.082932,10492.448391
19,CMEX1U5 CORP,CDS_REPL,133.155,CDS MEXICO 12/27 ICE,0.009459,126.800000,6.355000,4772.633397
